In [9]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
%matplotlib tk
# % matplotlib inline
#plt.rcParams['animation.html'] = 'jshtml'
from IPython.display import HTML

from copy import copy
import os
import sys
import imp
from functools import reduce
sys.path.append("/home/ful7rng/projects/transition/propagation")
from utils import pickle_load, get_npy, get_file
from utils.occ_map_utils import load_map, show_map, plot_occ_map, plot_trajectories, free_space
from utils.scene_utils import display_scenes_trajs, animate_scenes, get_scenes
from propagation.bofum import conditionalBOFUM, naiveBOFUM, BOFUMRealdata
from propagation.animation import TrackingAnimRealdata
from data_loader import get_map_crop


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
plt.ion()

In [6]:
from utils.plot_utils import plot_4d_tensor

def onclick(event, models, anim, fig_1, fig_2):
    ix, iy = event.xdata, event.ydata
    coords = np.floor(np.array([ix, iy]) / models[0].map_res).astype(int)
    print(coords)
    
    all_axes = [plot.axes for plot in anim.plots]
    
    for i, ax in enumerate(all_axes):

        # For infomation, print which axes the click was in
        if ax == event.inaxes:
            print "Click is in axes ax{}".format(i+1)
            break
            
    clicked = np.zeros_like(models[0].map)
    x, y = coords[0], coords[1]
    clicked[x, y] = 1
    for plot in anim.plots:
        plot.set_axes_data("occupancy_axes", clicked)
    anim.fig.canvas.draw()
    
    fig_1.clear()
    fig_2.clear()
    
    
    if models[i].kernels.ndim == 6:
        kernel = models[i].kernels[x, y]
        condi_prob = models[i].nn_probs[x, y]
    else:
        kernel = models[i].kernels
        condi_prob = models[i].kernels
        
    print(kernel.shape)
    print(condi_prob.shape)
    plot_4d_tensor(condi_prob, fig=fig_1)
    plot_4d_tensor(kernel, fig=fig_2)
    fig_1.canvas.draw()
    fig_2.canvas.draw()

In [7]:
diagonal = False
if diagonal:
    model_name = '23_ALL_MAPS_EIGHT_DIRECTIONS'
else:
    model_name = '01_JOINT_PROB_ALL_MAPS'

In [10]:
config_path = '/local/home/ful7rng/projects/transition/config.py'
config = imp.load_source('config', config_path)
conditional = False

In [11]:
from propagation.bofum import conditionalBOFUM, naiveBOFUM
from data_loader import get_map_crop

config_path = '/local/home/ful7rng/projects/transition/config.py'
maps, outputs = get_map_crop(config_path, num=1, dataset='train', condi_probs=True)
map_ = maps[0]
output = outputs[0]

(27119, 32, 32)


In [ ]:
fname = '/local/data/scenes/100_11/c3bo/05_10/2017-05-10-16-24-56_8.npy'
fname = '/local/data/scenes/100_11/c3bo/05_04/2017-05-04-19-07-38_10.npy'
scene= get_scenes(random_file=True,
                  min_time_interval=2,
                  max_time_interval=4,
                  file_name=fname)[0][0]
map_ = scene.static_map

In [ ]:
animate_scenes([BOFUMRealdata.scene_preprocessing(copy(scene))])

In [ ]:
laser_frequency = 12
min_time_interval = 3
num_steps = laser_frequency * min_time_interval

In [ ]:
models = []
force_predict = True
simulated_data = False
kwargs = dict(omega=0.1,
                  extent=7,
                  noise_var=0.1,
                  lost_prob=0,
                  verbose=False)


bofum_1 = conditionalBOFUM(map_, model_name,
                               name = 'BOFUM ',
                               simulated_data=simulated_data,
                               force_predict=True,
                               acceleration_interpretation=True,
                               conditional=conditional,
                               **kwargs)


bofum_2 = conditionalBOFUM(map_, model_name,
                               name = 'BOFUM w condi_prob blur',
                               simulated_data=simulated_data,
                               force_predict=False,
                               acceleration_interpretation=True,
                               conditional=conditional,
                               nn_probs=output,
                               **kwargs)


models.append(bofum_1)
models.append(bofum_2)
map(lambda model: model.initialize(scene), models)

blurred_kernels = blur_kernels_spatially(bofum.kernels)
bofum_1.kernels = blurred_kernels
blurred_condi = blur_kernels_spatially(bofum.conditional_probs)
bofum_2.conditional_probs = blurred_condi
bofum_2.construct_kernels()
blurred_condi = blur_kernels_spatially(bofum.conditional_probs, blur_extent=7, var=0.8)
bofum_3.conditional_probs = blurred_condi
bofum_3.construct_kernels()

In [ ]:

ani = TrackingAnimRealdata(models, num_steps, scene,
                                plot_map=True,
                                plot_seen=True)

fig_1 = plt.figure(figsize=(6, 5))
fig_2 = plt.figure(figsize=(6, 5))

ani.fig.canvas.mpl_connect('button_press_event', lambda event: onclick(event, models, ani, fig_1, fig_2))